<a href="https://colab.research.google.com/github/cyclic-reference/fast-ai-swift/blob/master/notebook_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmpkfsgdfpp/swift-install
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/saeta/Just
Fetching https://github.com/latenitesoft/NotebookExport
Completed resolution in 3.41s
Cloning https://github.com/latenitesoft/NotebookExport
Resolving https://github.com/latenitesoft/NotebookExport at 0.6.0
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.3
Cloning https://github.com/saeta/Just
Resolving https://github.com/saeta/Just at 0.7.3
[1/11] Compiling Path Extensions.swift
[2/11] Compiling Path Path+Attributes.swift
[3/11] Compiling Path Path+Codable.swift
[4/11] Compiling Path Path+CommonDirectories.swift
[5/11] Compiling Pat

In [0]:
// export
precedencegroup ExponentiationPrecedence {
    associativity: right
    higherThan: MultiplicationPrecedence
}
infix operator ** : ExponentiationPrecedence

precedencegroup CompositionPrecedence { associativity: left }
infix operator >| : CompositionPrecedence

In [0]:
//export
import Foundation
import Just
import Path


In [0]:
//export
public extension String {
    @discardableResult
    func shell(_ args: String...) -> String {
        let (task,pipe) = (Process(),Pipe())
        task.executableURL = URL(fileURLWithPath: self)
        (task.arguments,task.standardOutput) = (args,pipe)
        do    { try task.run() }
        catch { print("Unexpected error: \(error).") }

        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        return String(data: data, encoding: String.Encoding.utf8) ?? ""
    }
}

In [27]:
print("/bin/ls".shell("-lh"))

total 9.5M
drwxr-xr-x 1 root root 4.0K Aug 27 16:17 sample_data
drwxr-xr-x 4 root root 4.0K Sep  3 22:52 swift-install
-rw------- 1 root root 9.5M Sep  3 22:55 train-images-idx3-ubyte.gz



In [0]:
//export
public func downloadFile(_ url: String, dest: String? = nil, force: Bool = false) {
    let dest_name = dest ?? (Path.cwd/url.split(separator: "/").last!).string
    let url_dest = URL(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if !force && Path(dest_name)!.exists { return }

    print("Downloading \(url)...")

    if let cts = Just.get(url).content {
        do    {try cts.write(to: URL(fileURLWithPath:dest_name))}
        catch {print("Can't write to \(url_dest).\n\(error)")}
    } else {
        print("Can't reach \(url)")
    }
}

In [8]:
downloadFile("https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz")

In [0]:
//export
import TensorFlow

In [0]:
//export
protocol ConvertibleFromByte: TensorFlowScalar {
    init(_ d:UInt8)
}

In [0]:
//export
extension Float : ConvertibleFromByte {}
extension Int32 : ConvertibleFromByte {}

In [0]:
func loadMNISTData<T: ConvertibleFromByte>(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
    let split = training ? "train" : "t10k"
    let kind = labels ? "labels" : "images"
    let batch = training ? 60000 : 10000
    let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28])
    let dropK = labels ? 8 : 16
    let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
    let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
    let file = path/fname
    if !file.exists {
        downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
        "/bin/gunzip".shell("-fq", (path/"\(fname).gz").string)
    }
    let data = try! Data(contentsOf: URL(fileURLWithPath: file.string)).dropFirst(dropK)
    if labels { return Tensor(data.map(T.init)) }
    else      { return Tensor(data.map(T.init)).reshaped(to: shape)}
}

In [0]:
public func loadMNIST(path:Path, flat:Bool = false)
        -> (Tensor<Float>, Tensor<Int32>, Tensor<Float>, Tensor<Int32>) {
    try! path.mkdir(.p)
    return (
        loadMNISTData(training: true,  labels: false, path: path, flat: flat) / 255.0,
        loadMNISTData(training: true,  labels: true,  path: path, flat: flat),
        loadMNISTData(training: false, labels: false, path: path, flat: flat) / 255.0,
        loadMNISTData(training: false, labels: true,  path: path, flat: flat)
    )
}

In [0]:
//export
public let mnistPath = Path.cwd

In [38]:
let (xTraining, yTraining, xValidation, yValidation) = loadMNIST(path: mnistPath)
xTraining.shape

: ignored

expression produced error: error: /tmp/expr186-43f5df..swift:1:70: error: 'Path' is not a member type of 'Path'
Swift._DebuggerSupport.stringForPrintObject(Swift.UnsafePointer<Path.Path>(bitPattern: 0x7fb46d79f6e0)!.pointee)
                                                                ~~~~ ^

